# Notebook For Q&A OffLine Evaluation

## 1. Retrival Groundtruth : Create a set of possible questions for each Q&A

In [1]:
import sys
sys.path.append('../models/src')

# Now you can import hypno
from chains.hypnoq_chain import vector_store, groq_llm
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

import os
import ast
from tqdm import tqdm 
import pandas as pd

/home/codespace/.cache/pypoetry/virtualenvs/models-io_XXpQJ-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 24132.93it/s]


### loading the q&a vector database

In [4]:
# Access the document store
stored_docs = vector_store.docstore._dict

### a prompt for emulating test questions

In [3]:
emulation_prompt = """You emulate a potential client who is considering hypnotherapy services.
Formulate 5 questions this client might ask based on a FAQ record that is provided.
The record should contain the answer to the questions, and the questions should be complete and not too short.

The record:

question: {question}
answer: {text}

Provide the output as a valid JSON array without using code blocks, extra characters like asterisks (*), or numbering. The format should be exactly:

["question1", "question2", ..., "question5"]"""



emulation_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question","text"], template=emulation_prompt)
)

messages = [emulation_system_prompt]

emulation_tamplate = ChatPromptTemplate(
    input_variables=["question","text"], messages=messages
)

output_parser = StrOutputParser()

emulation_chain = emulation_tamplate | groq_llm | output_parser


### testing the prompt

In [4]:
answer = """Everyone is capable of change. Whether you will effect that change is largely down to you. Cognitive Hypnotherapy isn’t a miracle pill.
It’s a process of mental reprogramming. Our sessions will guide you to learn how to think, behave, and feel differently. 

Since I began practicing as a therapist, I’ve noticed some common traits among the clients who have made the most positive changes in their lives.
They are curious, willing to learn about themselves, and being prepared to put in the work. I can guide you. 
And I will support you always. But nobody can do it for you."""

reply = emulation_chain.invoke(dict(question='Q: Will hypnotherapy work for Me?',text=answer))
print(reply)

["Will hypnotherapy work for me even if I'm skeptical about its effectiveness?", 
"How much control will I have during the hypnotherapy sessions?", 
"What is expected from me in terms of effort and commitment for the therapy to be successful?", 
"Can cognitive hypnotherapy help me overcome habits that I've had for many years?", 
"What are the common traits of clients who have achieved positive changes through hypnotherapy?"]


### creating the ground truth dataset for testing the chatbot

In [5]:
val_dict = dict(doc_id=[], orig_quest=[], questions=[])

# Iterate over the documents with a progress bar
for _, doc in tqdm(stored_docs.items(), desc="Processing Documents"):

    quest, answer = doc.page_content.split('\n', 1)
    doc_id = doc.metadata.get('doc_id', None)
    reply = emulation_chain.invoke(dict(question=quest, text=answer))

    try:
        # Try to convert the string to an actual list
        actual_list = ast.literal_eval(reply)
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing the string: {e}")
        continue

    val_dict['doc_id'].append(doc_id)
    val_dict['orig_quest'].append(quest)
    val_dict['questions'].append(actual_list)
   

Processing Documents: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:19<00:00,  1.14s/it]

Error parsing the string: unterminated string literal (detected at line 6) (<unknown>, line 6)


In [6]:
df = pd.DataFrame.from_dict(val_dict)

# Explode the 'questions' column to create a row for each question
df_ground_truth = df.explode('questions')

# Reset the index if needed
df_ground_truth.reset_index(drop=True, inplace=True)

df_ground_truth

,doc_id,orig_quest,questions
0,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,What exactly is hypnosis and how does it work?
1,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,Is hypnosis a safe procedure?
2,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,Can you explain how hypnosis can be used to ma...
3,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,Are there any risks associated with going into...
4,34cdb806-9fd2-4b74-895a-134bfdb7bcac,What is hypnosis and is it safe?,How is hypnosis similar to day-dreaming and ca...
...,...,...,...
75,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,I feel quite slightly embarrassed to seek help...
76,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,Will my concerns and fears about seeking help ...
77,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,What are some of the barriers that I might pas...
78,ce3a25a9-cf78-44c3-b0dd-48bb8664c974,Q: I feel quite slightly embarrassed to seek h...,Is it common for people to have reservations o...


In [7]:
df_ground_truth.to_csv('ground-truth-data.csv', index=False)

## 2. RAG Evaluation : Create a set of LLM answers and the original answers

In [2]:
# Now you can import hypno
from chains.hypnoq_chain import qna_vector_chain

In [6]:
rag_dict = dict(doc_id=[], question=[], orig_answer=[], llm_answer=[])

# Iterate over the documents with a progress bar
for _, doc in tqdm(stored_docs.items(), desc="Processing Documents"):

    quest, answer = doc.page_content.split('\n', 1)
    doc_id = doc.metadata.get('doc_id', None)
    reply = qna_vector_chain.invoke({'question':quest})

    rag_dict['doc_id'].append(doc_id)
    rag_dict['question'].append(quest)
    rag_dict['orig_answer'].append(answer)
    rag_dict['llm_answer'].append(reply.content)

Processing Documents: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [02:57<00:00, 10.44s/it]


In [7]:
df_llm = pd.DataFrame.from_dict(rag_dict)

df_llm.head(5)

,doc_id,question,orig_answer,llm_answer
0,8cc35327-1cc6-4b18-b54d-9173f0579153,What is hypnosis and is it safe?,During hypnosis I use this state within you to...,Hypnosis is a state during which your unconsci...
1,636635e5-020e-4e8e-a153-6f77dbefeb29,What is Cognitive Hypnotherapy?,"Cognitive Hypnotherapy is a person-centered, f...","Cognitive Hypnotherapy is a person-centered, f..."
2,2023c157-d1b5-4d55-9bd1-beb8758345b8,My approach,Your treatment plan will be tailored to your s...,"Based on the provided context, the hypnotherap..."
3,d1e4f485-2609-4ace-a7bd-113382652c2a,Q: What do we mean by hypnotherapy?,People tend to think of hypnotherapy as someth...,Hypnotherapy is a process of mental reprogramm...
4,79ebc3c8-067b-49fc-a669-d66915eccd2f,Q: Will hypnotherapy work for Me?,Everyone is capable of change. Whether you wil...,The text states that everyone is capable of ch...


In [18]:
df_llm.to_csv('llm_results.csv', index=False)